In [ ]:
!pip install tensorflow keras-nlp pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.1/644.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 61.8 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [ ]:
!pip install -U tensorflow keras-nlp

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import time
import keras_nlp
import numpy as np
from pathlib import Path

# Function to load and preprocess data
def load_data(gsheet_id, sheet_name):
    """Load data from Google Sheets and preprocess it"""
    gsheet_url = f"https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    try:
        df = pd.read_csv(gsheet_url)
        df = df.iloc[:, 0:3]  # Keep only first 3 columns

        # Clean text content
        df['content'] = (df['content'].str.lower()
                        .str.replace('\r\n', ' ', regex=False)
                        .str.replace('\t', ' ', regex=False)
                        .str.replace('\n', ' ', regex=False)
                        .str.replace('       ', ' ', regex=False)
                        .str.strip())
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

def test_generation(model, prompts):
    """Test model generation with given prompts"""
    print("\nTesting generation...")
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        output = model.generate(prompt, max_length=200)
        print(f"Generated text: {output}")

def create_model(sequence_length=128):
    """Create and compile the GPT-2 model with preset configurations"""
    # Load the preset tokenizer and preprocessor
    preset_name = "gpt2_base_en"

    # Initialize preprocessor with preset
    preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
        preset_name,
        sequence_length=sequence_length
    )

    # Create backbone from preset
    backbone = keras_nlp.models.GPT2Backbone.from_preset(preset_name)

    # Initialize model with backbone and preprocessor
    model = keras_nlp.models.GPT2CausalLM(
        backbone=backbone,
        preprocessor=preprocessor
    )

    # Compile model
    learning_rate = keras.optimizers.schedules.PolynomialDecay(
        5e-4,
        decay_steps=500 * 15,  # adjust based on your dataset size and epochs
        end_learning_rate=0.0,
    )

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        weighted_metrics=["accuracy"],
    )

    return model

def prepare_dataset(texts, batch_size=8):
    """Prepare dataset for training"""
    dataset = tf.data.Dataset.from_tensor_slices(texts)
    dataset = dataset.batch(batch_size)
    dataset = dataset.cache()
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset.take(500)  # Limiting to 500 batches as in original code

def save_model(model, base_path="./"):
    """Save model weights and configuration"""
    try:
        # Create directory if it doesn't exist
        Path(base_path).mkdir(parents=True, exist_ok=True)

        # Save full model with .keras extension
        model.save(f"{base_path}/model.keras")

        # Save weights separately
        weights_path = Path(base_path) / "model_weights"
        weights_path.mkdir(exist_ok=True)
        model.save_weights(f"{weights_path}/weights.keras")

        print(f"Model saved successfully to {base_path}")

    except Exception as e:
        print(f"Error saving model: {e}")

        # Try alternative h5 format if keras format fails
        try:
            print("Attempting to save in h5 format...")
            model.save(f"{base_path}/model.h5")
            model.save_weights(f"{weights_path}/weights.h5")
            print("Model saved successfully in h5 format")
        except Exception as e2:
            print(f"Error saving in h5 format: {e2}")

def main():
    # Configuration
    GSHEET_ID = "191YRBsdUEGtgXvWl428L9xt48Ah5zcyoRK1pvNhUDPM"
    SHEET_NAME = "Sheet2"
    NUM_EPOCHS = 15

    # Test prompts
    test_prompts = [
        "My trip to Kuala Lumpur was so annoying",
        "raja dan permaisuri"
    ]

    # Create base model and test it first
    print("Creating base model...")
    base_model = keras_nlp.models.GPT2CausalLM.from_preset("gpt2_base_en")

    print("\nTesting base model (before fine-tuning):")
    test_generation(base_model, test_prompts)

    # Load and preprocess data
    print("\nLoading data for fine-tuning...")
    df = load_data(GSHEET_ID, SHEET_NAME)
    if df is None:
        return

    # Create and compile model for fine-tuning
    print("\nCreating model for fine-tuning...")
    model = create_model()

    # Prepare dataset
    print("Preparing dataset...")
    train_ds = prepare_dataset(df['content'])

    # Train model
    print("Starting fine-tuning...")
    start_time = time.time()
    model.fit(train_ds, epochs=NUM_EPOCHS)
    end_time = time.time()
    print(f"Training completed in {end_time - start_time:.2f} seconds")

    # Test fine-tuned model
    print("\nTesting fine-tuned model:")
    test_generation(model, test_prompts)

    # Save model
    print("\nSaving fine-tuned model...")
    save_model(model)

if __name__ == "__main__":
    main()

Creating base model...

Testing base model (before fine-tuning):

Testing generation...

Prompt: My trip to Kuala Lumpur was so annoying
Generated text: My trip to Kuala Lumpur was so annoying, I decided to go with my friend to a local hotel. I was very nervous, but I didn't have any issues. We arrived at the hotel in a hurry, and it's a very nice hotel, so I went to check on my luggage, but the hotel was a little bit crowded. I didn't want to wait too long, so I went with my friend. I got to the hotel and I saw the room that was next to the hotel and it was very comfortable. I was happy and I didn't feel like I was in a bad mood. I went in the lobby with my friend and we were very happy with the room. We went out to the bathroom and I was very pleased with how comfortable it was, but it wasn't very comfortable for my body. It was so hot, so hot, I didn't even think about it. I didn't want to get sweaty and I didn't feel like I was sweaty

Prompt: raja dan permaisuri
Generated text: ra

Error saving model: The filename must end in `.weights.h5`. Received: filepath=model_weights/weights.keras
Attempting to save in h5 format...
Error saving in h5 format: The filename must end in `.weights.h5`. Received: filepath=model_weights/weights.h5


In [ ]:
!pip freeze > requirements.txt

In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_nlp
from pathlib import Path

def load_saved_model(model_path="./model.keras"):
    """
    Load a saved GPT-2 model.
    Args:
        model_path: Path to the saved model file
    Returns:
        Loaded model ready for inference
    """
    try:
        # Load the full model
        model = keras.models.load_model(model_path)
        print(f"Model successfully loaded from {model_path}")
        return model
    except Exception as e:
        print(f"Error loading model from {model_path}: {e}")

        # Try loading from h5 format if keras format fails
        try:
            h5_path = str(Path(model_path).parent / "model.h5")
            print(f"Attempting to load from h5 format: {h5_path}")
            model = keras.models.load_model(h5_path)
            print("Model successfully loaded from h5 format")
            return model
        except Exception as e2:
            print(f"Error loading from h5 format: {e2}")
            return None

def generate_text(model, prompt, max_length=200, temperature=0.7):
    """
    Generate text using the loaded model.
    Args:
        model: Loaded GPT-2 model
        prompt: Input text to generate from
        max_length: Maximum length of generated text
        temperature: Controls randomness (higher = more random)
    Returns:
        Generated text
    """
    try:
        output = model.generate(
            prompt,
            max_length=max_length,
            temperature=temperature
        )
        return output
    except Exception as e:
        print(f"Error generating text: {e}")
        return None

# Example usage
def main():
    # Load the model
    model = load_saved_model()
    if model is None:
        return

    # Test some prompts
    test_prompts = [
        "My trip to Kuala Lumpur was so annoying",
        "raja dan permaisuri",
        "The weather in Malaysia is"
    ]

    print("\nGenerating text from test prompts:")
    for prompt in test_prompts:
        print(f"\nPrompt: {prompt}")
        generated_text = generate_text(model, prompt)
        print(f"Generated: {generated_text}")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 393 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model successfully loaded from ./model.keras

Generating text from test prompts:

Prompt: My trip to Kuala Lumpur was so annoying
Error generating text: CausalLM.generate() got an unexpected keyword argument 'temperature'
Generated: None

Prompt: raja dan permaisuri
Error generating text: CausalLM.generate() got an unexpected keyword argument 'temperature'
Generated: None

Prompt: The weather in Malaysia is
Error generating text: CausalLM.generate() got an unexpected keyword argument 'temperature'
Generated: None
